In [75]:
from ipywidgets import interact
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time

In [128]:
from sequence.kernel.timeline import Timeline
from sequence.topology.node import Node
from sequence.components.light_source import SPDCSource
from sequence.kernel.process import Process
from sequence.kernel.event import Event
from sequence.utils import log
import logging
logger = logging.getLogger()
from sequence.kernel.timeline import Timeline
#logger.setLevel(logging.DEBUG)


class Counter:
    def __init__(self):
        self.count = 0
        self.time = 0

    def trigger(self, detector, info):
        self.count += 1
        self.time = info['time']
        



class SenderNode(Node):
    def __init__(self, name, timeline):
        super().__init__(name, timeline)
        self.n_pairs = 0
        
        SPDCSource_name = name + ".spdcsource"
        self.spdcsource = SPDCSource(SPDCSource_name, timeline, frequency=8e7, mean_photon_num=5)
        self.add_component(self.spdcsource)
        
        self.spdcsource.add_receiver(self)
        self.spdcsource.add_receiver(self)
        self.periods = []
        self.counts = []
        
    def start(self, period,NUM_TRIALS):
        states = [[[complex(1), complex(1)]]]
        process1 = Process(self.spdcsource, "emit",states)
        process2 = Process(self, "init_counter",[])
        for i in range(NUM_TRIALS):
            event1 = Event(i * period, process1)
            event2 = Event(i * period + 0.9*period, process2)
            self.periods.append(i * period)
            
            self.timeline.schedule(event1)
            self.timeline.schedule(event2)
    def init_counter(self):
        self.counts.append(self.n_pairs//2)
        self.n_pairs = 0



    def get(self, photon: "Photon", **kwargs):
        self.n_pairs +=1


        
def test(sim_time,mean_photon_num,nb_of_trials):
    PS_PER_MS = 1e9
    
    tl = Timeline(sim_time * PS_PER_MS)
    my_source = SenderNode("my_source",tl)
    my_source.set_seed(0)
    FREQUENCY = 1e9
    NUM_TRIALS= nb_of_trials
    my_source.spdcsource.mean_photon_num = mean_photon_num
    tl.init()
    period = int(1e12 / FREQUENCY)
    my_source.start(period,NUM_TRIALS)
    tl.run()
    
    plt.figure()
    
    x=my_source.periods
    y=my_source.counts
    
    plt.plot(x,y)
    #plt.plot(data,range(1, len(data) + 1),marker ='x')
    df = pd.DataFrame({"period": x, "counts":y})
    
    data_count = df[["counts"]].apply(pd.value_counts)
    
    
    data_count.sort_index().plot(kind="bar")
    plt.show()

In [129]:
interactive_plot = interact(test, sim_time=(100, 1000, 100),mean_photon_num=(0, 10, 1), nb_of_trials=(0, 500, 1))
interactive_plot

interactive(children=(IntSlider(value=500, description='sim_time', max=1000, min=100, step=100), IntSlider(val…

<function __main__.test(sim_time, mean_photon_num, nb_of_trials)>